In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/My Drive/ProyectoTesis/AllFeatureData.pkl'


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
allData = pd.read_pickle(file_path)

In [ ]:
featureData = allData.copy()

In [ ]:
featureData

In [ ]:
features = allData.columns[:-1]
features

In [ ]:
for feature in features:
  featureData[feature] = featureData[feature].apply(lambda x: np.mean(x))

featureData.head()

In [ ]:
import pickle as pkl

In [ ]:
''' path = '/content/drive/My Drive/ProyectoTesis/MeanDataNoFilter.pkl' '''

In [ ]:
''' # Save the object to a file
with open(path, 'wb') as file:  # Open the file in write-binary mode
    pkl.dump(featureData, file) '''

# Autogluon

In [ ]:
%pip install autogluon.tabular

In [ ]:
from autogluon.tabular import TabularPredictor


In [ ]:
path_to_predictor = "/content/drive/My Drive/ProyectoTesis/Autogluon/Models"

In [ ]:
#predictor = TabularPredictor.load(path_to_predictor)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data = featureData.copy()

In [ ]:
data.head()

In [ ]:
X = data.drop('Label', axis=1)
y = data['Label']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
data['Label'] = y_encoded

In [ ]:
data.head()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
for i in range(len(features)):
  data[features[i]] = X_scaled.T[i]

In [ ]:
allData.head()

In [ ]:
import pickle as pkl
with open('/content/drive/My Drive/ProyectoTesis/Scaler.pkl', 'wb') as file:
  pkl.dump(scaler, file)

In [ ]:
data.head()

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=y)

## Hasta Aqui llega Cargar Datos

In [ ]:
from autogluon.tabular import TabularPredictor

In [ ]:
path="/content/drive/My Drive/ProyectoTesis/Autogluon/Modelos2"

In [ ]:

label = 'Label'

predictor = TabularPredictor(label=label, problem_type='binary', path="/content/drive/My Drive/ProyectoTesis/Autogluon/Modelos3", verbosity=2, eval_metric='f1_weighted', log_to_file=True)

In [ ]:
predictor.fit(
    train_data=train_data,
    time_limit=5400, 
    ag_args_fit={'use_gpu': True},
    excluded_model_types=['NN_TORCH', 'FASTAI'],
    dynamic_stacking=False,

)

## Cargar predictor

In [ ]:
predictor = TabularPredictor.load(path)

In [ ]:
feature_importance = predictor.feature_importance(data=test_data)
print(feature_importance)

In [ ]:
importance_df = pd.DataFrame({'feature': feature_importance.index, 'importance': feature_importance['importance']})

In [ ]:
importance_df = importance_df.sort_values(by='importance', ascending=False)

colors = sns.color_palette("rocket", 3)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importance_df, color=colors[2])
plt.title('')
plt.xlabel('Importancia de la característica')
plt.ylabel('Características')
plt.show()

In [ ]:
predictor.evaluate(test_data, detailed_report=True)

In [ ]:
predictor.leaderboard()

In [ ]:
predictor.model_names()

In [ ]:
for model_name in predictor.leaderboard()['model']:
    model = predictor._trainer.load_model(model_name)
    print(f"Model: {model_name}")
    print(model.params)
    print("-" * 60)


In [ ]:
models = {model_name: predictor._trainer.load_model(model_name) for model_name in predictor.model_names() }

In [ ]:
models

In [ ]:
leaderboard = predictor.leaderboard(test_data, extra_metrics=['f1_weighted', 'recall', 'precision', ''])
print(leaderboard.columns)

In [ ]:
leaderboard_df = leaderboard

In [ ]:
leaderboard_df

In [ ]:
X_test = test_data.drop("Label", axis=1)
y_test = test_data["Label"]

In [ ]:
def GetConfusionMatrix(model_name, test_data=X_test, real_data=y_test):
  from sklearn.metrics import confusion_matrix
  y_pred = predictor.predict(test_data, model=model_name)
  conf_matrix = confusion_matrix(real_data, y_pred)
  return conf_matrix


In [ ]:
def GetConfusionMatrix2(model_name, model, indexes):
  from sklearn.metrics import confusion_matrix
  test_data = allData[]
  y_pred = model.predict(test_data)
  conf_matrix = confusion_matrix(real_data, y_pred)
  return conf_matrix

In [ ]:
def MetricsDfConverter(classes, metrics):
  cat1, cat2, macro = metrics
  columns = ["Precision", "Recall", "F1-Score"]
  df = pd.DataFrame(metrics, classes, columns)

  for column in columns:
    df[column] = df[column].apply(lambda x: f"{x:.4f}")

  return df

In [ ]:
def MetricsCalculator(confusion_matrix, model=None):
  bonafide, spoof = confusion_matrix

  ps = spoof[1]/(spoof[1]+bonafide[1])
  rs = spoof[1]/sum(spoof)
  f1s = 2*(ps*rs)/(ps+rs)

  s_metrics = [ps,rs,f1s]


  pb = bonafide[0]/(bonafide[0]+spoof[0])

  if np.isnan(pb):
    pb = 0

  rb = bonafide[0]/sum(bonafide)
  f1b = 2*(pb*rb)/(pb+rb)

  if np.isnan(f1b):
    f1b = 0

  b_metrics = [pb,rb,f1b]


  macro_p = (ps+pb)/2
  macro_r = (rs + rb)/2
  macro_f1 = (f1s + f1b)/2

  macro_metrics = [macro_p, macro_r, macro_f1]

  classes = ["spoof", "bonafide", "Macro-Avg"]
  metrics = [s_metrics, b_metrics, macro_metrics]

  print(f"\n************ {model} ************")
  print(f"Accuracy: {(spoof[1]+bonafide[0])/(sum(bonafide)+sum(spoof)):.4f}\n")
  print(MetricsDfConverter(classes, metrics))
  print("\n**********************************\n")

  return macro_metrics



In [ ]:
import time

def MeasureResponseTime(loaded_models, flag=0, models=None):
  for model_name in loaded_models:
    ttime = 0
    for i in range(1000):
      start_time = time.time()

      first_instance = X_test[i:i+1]
      if flag==0:
        y_pred = predictor.predict(first_instance, model=model_name)
      else: y_pred = models[model_name].predict(first_instance)

      end_time = time.time()

      execution_time = end_time - start_time
      ttime += execution_time
      '''       if i %100 ==0:
              print(model_name, execution_time) '''

    print(f"Model: {model_name}\t\t Execution time: {ttime/1000:.6f} seconds")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#categories = ['Kernel radial', 'Kernel polinomial', 'Kernel sigmoide']
#modelos = [svm_rad, svm_pol, svm_sig]


# Data
def GraphDifferences(categories, modelos, hvalues=None):
  categories = categories
  colors = sns.color_palette("rocket", 3)
  modelos = modelos
  metricas = [[modelo[i] for modelo in modelos] for i in range(3)]

  precision, recall, f1_score = metricas  

  x = np.arange(len(categories))  

  width = 0.2

  fig, ax = plt.subplots(figsize=(8, 6))

  bars1 = ax.bar(x - width, precision, width, label='Precision', color=colors[0])
  bars2 = ax.bar(x, recall, width, label='Recall', color=colors[1])
  bars3 = ax.bar(x + width, f1_score, width, label='F1-Score', color=colors[2])
  loc='lower left'
  if hvalues:
    plt.axhline(hvalues[0], color='red', linestyle='--',  alpha=0.7, linewidth=1.5, label=f'Precision Humanos: {hvalues[0]:.2f}')
    plt.axhline(hvalues[1], color='green', linestyle='--', alpha=0.7, linewidth=1.5, label=f'Recall Humanos: {hvalues[1]:.2f}')
    plt.axhline(hvalues[2], color='blue', linestyle='--',alpha=0.7, linewidth=1.5, label=f'F1-score Humanos: {hvalues[2]:.2f}')
    loc = 'lower right'
  ax.set_xlabel('Modelos')
  ax.set_ylabel('Puntaje')
  ax.set_xticks(x)
  ax.set_xticklabels(categories, rotation=90)
  ax.legend(loc=loc, framealpha=0.95)

  plt.tight_layout()
  plt.show()


In [ ]:
conf_matrices = []

In [ ]:
for model_name in predictor.model_names():
  conf_matrices.append((GetConfusionMatrix(model_name), model_name))


In [ ]:
conf_matrices

In [ ]:
leaderboard_sorted_by_f1 = leaderboard.sort_values(by='eval_metric', ascending=False)
print(leaderboard_sorted_by_f1)


In [ ]:
model_metrics = []

In [ ]:
for model in conf_matrices:
  model_metrics.append(MetricsCalculator(*model))

In [ ]:
GraphDifferences(predictor.model_names()[:-1], model_metrics[:-1])

In [ ]:
MeasureResponseTime(predictor.model_names())

In [ ]:
start_time = time.time()

y_pred = predictor.predict(X_test, model="KNeighborsDist")

end_time = time.time()
execution_time = end_time - start_time
print(f"Model: KNeighborsDist \t\t Execution time: {execution_time} seconds")

start_time = time.time()

y_pred = predictor.predict(X_test, model="RandomForestGini")

end_time = time.time()
execution_time = end_time - start_time
print(f"Model: RandomForestGini \t\t Execution time: {execution_time} seconds")

In [ ]:
len(X_test)

# Baseline models

In [ ]:
X_train = train_data.drop('Label', axis=1)
y_train = train_data['Label']

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

start_time = time.time()
# 1. Most Frequent Class Classifier
most_frequent_clf = DummyClassifier(strategy="most_frequent")
most_frequent_clf.fit(X_train, y_train)

end_time = time.time()
training_time = end_time-start_time

print(f"Model: BaseLine MostFrequent \t\t Execution time: {training_time} seconds")

In [ ]:
# 2. Random Classifier
start_time = time.time()

random_clf = DummyClassifier(strategy="uniform")
random_clf.fit(X_train, y_train)

end_time = time.time()
training_time = end_time-start_time

print(f"Model: BaseLine Random \t\t Execution time: {training_time} seconds")

In [ ]:
#3. Logisitic Regression
start_time = time.time()


lr_clf = LogisticRegression(random_state=0)
lr_clf.fit(X_train, y_train)

end_time = time.time()
training_time = end_time-start_time
print(f"Model: BaseLine Random \t\t Execution time: {training_time} seconds")

In [ ]:
# Predictions
y_pred_most_frequent = most_frequent_clf.predict(X_test)
y_pred_random = random_clf.predict(X_test)
y_pred_lr = lr_clf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# Evaluation - Accuracy and Confusion Matrix
print("Most Frequent Class Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_most_frequent))
print("Confusion Matrix:\n", cf_mf := confusion_matrix(y_test, y_pred_most_frequent))

print("\nRandom Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_random))
print("Confusion Matrix:\n", cf_r := confusion_matrix(y_test, y_pred_random))

print("\nRandom Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Confusion Matrix:\n", cf_lr := confusion_matrix(y_test, y_pred_lr))

In [ ]:
baseline_models = [most_frequent_clf, random_clf, lr_clf]
model_names = ['Most Frequent', 'Random', 'Logistic Regression']
cf_ms = [cf_mf, cf_r, cf_lr]
cf_ms = zip(cf_ms, model_names)

In [ ]:
for cf, model_name in cf_ms:
  MetricsCalculator(cf, model_name)

In [ ]:
bs_models = {model_names[i]:baseline_models[i] for i in range(len(model_names))}

In [ ]:
MeasureResponseTime(model_names, flag=1, models=bs_models)

# Pruebas con el set de datos de reconocimiento humano

In [ ]:
indexes_for_human_dataset = [42253, 44952, 70840, 115974, 164782, 170033, 172663, 253854, 269530, 290934]

In [ ]:
data.head()

In [ ]:
featureData[featureData['zcr']==]

In [ ]:
allData.iloc[indexes_for_human_dataset]['zcr']

In [ ]:
allData.tail()

In [ ]:
human_audios = data.iloc[indexes_for_human_dataset]
human_audios

In [ ]:
human_data_conf_matrices = []

In [ ]:
for model_name in predictor.model_names():
  human_data_conf_matrices.append((GetConfusionMatrix(model_name, human_audios[human_audios.columns[:-1]], human_audios['Label']), model_name))

In [ ]:
import pickle as pkl

In [ ]:
human_data_conf_matrices = human_data_conf_matrices[:-1]

In [ ]:
best_lstm = (np.array([[3, 2],
                        [0, 5]]), "LSTM LR=0.0005")

In [ ]:
best_svm = (np.array([[5, 0],
                    [0, 5]]), "SVM RBF C=1000.0")

In [ ]:
human_data_conf_matrices.append(best_lstm)
human_data_conf_matrices.append(best_svm)

In [ ]:
human_data_conf_matrices

In [ ]:
human_model_metrics = []

In [ ]:
GraphDifferences(predictor.model_names()[:-1], model_metrics[:-1])

In [ ]:
for model in human_data_conf_matrices:
  human_model_metrics.append((model[1], MetricsCalculator(*model)))

In [ ]:
human_model_metrics

In [ ]:
cats = [i[0] for i in human_model_metrics]
models_all = [i[1] for i in human_model_metrics]

In [ ]:
humanos = (np.array([[261, 209],[210, 260]]), "Reconocimiento humano")

In [ ]:
confusion_matrix, model = humanos
confusion_matrix, model

In [ ]:
metricas_de_humanos = MetricsCalculator(*humanos)

In [ ]:
GraphDifferences(cats ,models_all, hvalues = metricas_de_humanos)

In [ ]:
#Matriz de confusión del exámen
# array([[261, 209],
#        [210, 260]])

#Matrices de confusión del examen para modelos lstm
# [(array([[3, 2],
#          [0, 5]]),
#   0.0001),
#  (array([[3, 2],
#          [0, 5]]),
#   0.0005),
#  (array([[3, 2],
#          [0, 5]]),
#   0.001),
#  (array([[2, 3],
#          [0, 5]]),
#   0.005),
#  (array([[2, 3],
#          [0, 5]]),
#   0.01)]

#Matrices de consusión del examen para modelos svm con kernel radial
# [(array([[0, 5],
#          [0, 5]]),
#   0.1),
#  (array([[0, 5],
#          [0, 5]]),
#   1.0),
#  (array([[2, 3],
#          [0, 5]]),
#   10.0),
#  (array([[4, 1],
#          [0, 5]]),
#   100.0),
#  (array([[5, 0],
#          [0, 5]]),
#   1000.0)]

# Pruebas con filtros

## Filtro media

In [ ]:
prueba_filtro_media = allData.copy()

In [ ]:
prueba_filtro_media.head()

In [ ]:
features_media = prueba_filtro_media.columns[:-1]

In [ ]:
features_media

In [ ]:
import numpy as np
from scipy.ndimage import uniform_filter1d

window_size = 5

In [ ]:
def MeanFilter(row, window_size):
  smoothed_features = uniform_filter1d(row, size=window_size, axis=0)
  return smoothed_features

In [ ]:
def GetFeatureMean(data, features):
  for feature in features:
    data[feature] = data[feature].apply(lambda x: np.mean(x))



In [ ]:
prueba = prueba_filtro_media.copy()

In [ ]:
prueba['zcr'][0]

In [ ]:
for feature in features_media:
  prueba[feature] = prueba[feature].apply(MeanFilter, window_size=window_size)

In [ ]:
np.mean(prueba['zcr'][0])

In [ ]:
GetFeatureMean(prueba, features_media)

In [ ]:
prueba.head()

In [ ]:
featureData.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

# Histogram before filtering
plt.figure(figsize=(10, 6))
plt.hist(prueba['zcr'], bins=30, alpha=0.5, color='skyblue', edgecolor='black', label='Before Filter')
plt.hist(featureData['zcr'], bins=30, alpha=0.5, color='salmon', edgecolor='black', label='After Filter')

# Add labels and legend
plt.title('Comparison of ZCR Histograms Before and After Mean Filter')
plt.xlabel('ZCR Value')
plt.ylabel('Frequency')
plt.legend(loc='upper right')


In [ ]:
prueba['zcr'].hist()
featureData['zcr'].hist()

In [ ]:
len(prueba)

In [ ]:
len()